In [ ]:
# =============================================================================
# # BÖLÜM 1: Gerekli Kütüphanelerin Yüklenmesi
# =============================================================================
# Bu hücrede, projemiz boyunca ihtiyaç duyacağımız tüm kütüphaneleri yüklüyoruz.
# - numpy ve pandas: Veri manipülasyonu ve analizi için temel araçlar.
# - matplotlib: Sonuçları görselleştirmek için kullanılır.
# - cv2 (OpenCV): Görüntü işleme operasyonları için gereklidir.
# - tensorflow ve keras: Derin öğrenme modelini oluşturmak, eğitmek ve değerlendirmek için kullanılır.
# - sklearn: Veri setini eğitim ve doğrulama olarak ayırmak için kullanılır.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import logging
import cv2
import time
from tqdm.notebook import tqdm
from scipy.signal import resample
from sklearn.model_selection import train_test_split

# TensorFlow ve Keras kütüphanelerinden gerekli modüller
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print("TensorFlow Sürümü:", tf.__version__)

In [ ]:
# =============================================================================
# # BÖLÜM 2: Yapılandırma ve Loglama Ayarları
# =============================================================================
# Bu hücre, projenin temel hiperparametrelerini ve ayarlarını merkezi bir yerde toplar.
# Bu sayede, farklı denemeler yapmak için sadece bu bölümü değiştirmek yeterli olacaktır.

# Loglama ayarları, kod çalışırken önemli bilgileri ve uyarıları takip etmemizi sağlar.
log_format = '%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_format)

class Config:
    """
    Tüm hiperparametreleri ve yapılandırma ayarlarını içeren sınıf.
    DEBUG_MODE: True ise, daha küçük bir veri alt kümesi ile hızlı denemeler yapılır.
    """
    # Genel Ayarlar
    DEBUG_MODE = True  # Hızlı deneme modu
    SEED = 42          # Tekrarlanabilir sonuçlar için rastgelelik tohumu

    # Görüntü ve Sinyal Boyutları
    IMG_HEIGHT = 128
    IMG_WIDTH = 512
    SEQ_LENGTH = 512   # Modelin üreteceği zaman serisi uzunluğu

    # Eğitim Hiperparametreleri
    BATCH_SIZE = 32
    EPOCHS_HEAD = 10 if not DEBUG_MODE else 4  # Sadece üst katmanların eğitimi için
    EPOCHS_FULL = 20 if not DEBUG_MODE else 6  # Tüm modelin ince ayarı için
    LEARNING_RATE_HEAD = 1e-3
    LEARNING_RATE_FULL = 1e-4 # İnce ayar için daha düşük bir öğrenme oranı
    DROPOUT_RATE = 0.3 # Ezberlemeyi önlemek için dropout oranı

    # Veri Kümesi Boyutları (DEBUG_MODE için)
    TRAIN_SAMPLES = 100
    VAL_SAMPLES = 20

# Yapılandırma sınıfından bir nesne oluşturuluyor
CONFIG = Config()
LEAD_NAMES = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

# Rastgelelik tohumunu ayarlayarak deneylerin tekrarlanabilirliğini sağlıyoruz
np.random.seed(CONFIG.SEED)
tf.random.set_seed(CONFIG.SEED)

logging.info(f"Hızlı mod (DEBUG_MODE) {'AÇIK' if CONFIG.DEBUG_MODE else 'KAPALI'}.")

# Önceden eğitilmiş model ağırlıklarının yolu
EFFICIENTNET_WEIGHTS_PATH = '/kaggle/input/keras-applications-models/EfficientNetB0.h5'
if not os.path.exists(EFFICIENTNET_WEIGHTS_PATH):
    raise FileNotFoundError(
        "EfficientNet ağırlık dosyası bulunamadı! Lütfen sağdaki menüden '+ Add Data' "
        "düğmesine basarak 'Keras Applications Models' veri setini notebook'unuza eklediğinizden emin olun."
    )

In [ ]:
# =============================================================================
# # BÖLÜM 3: Veri Yükleme ve Bölme
# =============================================================================
# Bu bölümde, yarışmanın sağladığı 'train.csv' ve 'test.csv' dosyalarını yüklüyoruz.
# Eğitim verisini, modelin performansını objektif bir şekilde ölçebilmek için
# eğitim (training) ve doğrulama (validation) setlerine ayırıyoruz.
# Ayırma işlemi 'id' bazında yapılarak aynı hastaya ait EKG'lerin farklı setlere dağılması engellenir.

logging.info("Yarışma verileri yükleniyor...")
data_dir = "/kaggle/input/physionet-ecg-image-digitization"

try:
    # Tüm eğitim verisini içeren CSV dosyasını oku
    full_train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
    test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
    
    # Benzersiz hasta ID'lerini al
    unique_ids = full_train_df['id'].unique()
    
    # Hasta ID'lerini %85 eğitim, %15 doğrulama olacak şekilde ayır
    train_ids, val_ids = train_test_split(unique_ids, test_size=0.15, random_state=CONFIG.SEED)
    
    # DataFrame'leri bu ID'lere göre filtrele
    train_df = full_train_df[full_train_df['id'].isin(train_ids)].copy()
    val_df = full_train_df[full_train_df['id'].isin(val_ids)].copy()
    
    # DEBUG_MODE aktif ise, veri setlerinden küçük bir örneklem al
    if CONFIG.DEBUG_MODE:
        train_df = train_df.sample(n=CONFIG.TRAIN_SAMPLES, random_state=CONFIG.SEED).reset_index(drop=True)
        val_df = val_df.sample(n=CONFIG.VAL_SAMPLES, random_state=CONFIG.SEED).reset_index(drop=True)

    logging.info(f"Eğitim verisi boyutu: {len(train_df)}, Doğrulama verisi boyutu: {len(val_df)}")

except Exception as e:
    logging.error(f"Veri yüklenirken/bölünürken hata oluştu: {e}")
    # Hata durumunda boş DataFrame'ler oluştur
    train_df, val_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [ ]:
# =============================================================================
# # BÖLÜM 4: Veri Hattı (Data Pipeline) Fonksiyonları
# =============================================================================
# Bu hücre, EKG görüntülerinden ve sinyal dosyalarından veri okuyup,
# modele uygun hale getiren fonksiyonları içerir. TensorFlow'un `tf.data` API'si
# kullanılarak verimli bir veri yükleme hattı oluşturulur.

def get_lead_images_from_file(image_path):
    """
    Bir EKG görüntüsünden 12 derivasyonun (lead) her birinin görüntüsünü kırparak çıkarır.
    
    Args:
        image_path (str): EKG görüntüsünün dosya yolu.
        
    Returns:
        list: 12 adet derivasyon görüntüsü içeren bir liste veya hata durumunda None.
    """
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None: 
            logging.warning(f"Görüntü okunamadı veya dosya bozuk: {image_path}")
            return None
        
        _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        height, width = thresh.shape
        
        lead_boxes = [
            (140, 205, 120, 620), (270, 335, 120, 620), (140, 205, 620, 1120), 
            (205, 270, 620, 1120), (270, 335, 620, 1120), (140, 205, 1120, 1620), 
            (205, 270, 1120, 1620), (270, 335, 1120, 1620), (140, 205, 1620, 2120), 
            (205, 270, 1620, 2120), (270, 335, 1620, 2120)
        ]
        rhythm_strip = thresh[400:480, 120:width-120]
        
        lead_images = [thresh[y1:y2, x1:x2] for y1, y2, x1, x2 in lead_boxes]
        lead_images.insert(1, rhythm_strip)
        return lead_images
    except Exception as e:
        logging.error(f"get_lead_images_from_file içinde beklenmedik hata: {e} - Dosya: {image_path}")
        return None

def process_path(image_path, signal_path=None):
    """
    Tek bir veri örneğini (görüntü ve sinyal) işler.
    Gelen 'image_path' ve 'signal_path' girdilerinin türünü kontrol ederek hem
    TensorFlow tensörleri hem de normal Python string'leri ile çalışabilir.
    """
    # Girdi türünü kontrol et: tf.Tensor ise Python string'e çevir.
    if isinstance(image_path, tf.Tensor):
        img_p_str = image_path.numpy().decode('utf-8')
    else:
        img_p_str = image_path

    # Sinyal yolu için de aynı kontrolü yap.
    if signal_path is not None and isinstance(signal_path, tf.Tensor):
        sig_p_str = signal_path.numpy().decode('utf-8')
    else:
        sig_p_str = signal_path

    lead_images_raw = get_lead_images_from_file(img_p_str)
    
    if lead_images_raw is None:
        return tf.zeros((12, CONFIG.IMG_HEIGHT, CONFIG.IMG_WIDTH, 3)), tf.zeros((12, CONFIG.SEQ_LENGTH))
    
    processed_images = []
    for img in lead_images_raw:
        if img.shape[0] == 0 or img.shape[1] == 0: 
            img = np.zeros((CONFIG.IMG_HEIGHT, CONFIG.IMG_WIDTH), dtype=np.uint8)
        img_resized = tf.image.resize(img[..., tf.newaxis], [CONFIG.IMG_HEIGHT, CONFIG.IMG_WIDTH])
        img_3_channel = tf.image.grayscale_to_rgb(img_resized)
        processed_images.append(img_3_channel)
    images_tensor = tf.stack(processed_images)
    
    if sig_p_str is not None:
        true_ts_df = pd.read_csv(sig_p_str)
        true_ts_df.ffill(inplace=True); true_ts_df.bfill(inplace=True)
        resampled_signals = [resample(true_ts_df[lead].values.astype(np.float32), CONFIG.SEQ_LENGTH) for lead in LEAD_NAMES]
        signals_tensor = tf.stack(resampled_signals)
        return images_tensor, signals_tensor
    else:
        signals_tensor = tf.zeros((12, CONFIG.SEQ_LENGTH))
        return images_tensor, signals_tensor

def create_dataset(df, is_train=True):
    """
    Verilen DataFrame'den bir tf.data.Dataset nesnesi oluşturur.
    """
    image_paths = [f"{data_dir}/train/{rec_id}/{rec_id}-0001.png" for rec_id in df['id']]
    signal_paths = [f"{data_dir}/train/{rec_id}/{rec_id}.csv" for rec_id in df['id']]
    
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, signal_paths))

    def py_func_wrapper(img_p, sig_p):
        images, signals = tf.py_function(process_path, [img_p, sig_p], [tf.float32, tf.float32])
        images.set_shape([12, CONFIG.IMG_HEIGHT, CONFIG.IMG_WIDTH, 3])
        signals.set_shape([12, CONFIG.SEQ_LENGTH])
        return images, signals

    dataset = dataset.map(py_func_wrapper, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_train: 
        dataset = dataset.shuffle(buffer_size=1000, seed=CONFIG.SEED)
        
    dataset = dataset.unbatch()
    dataset = dataset.batch(CONFIG.BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset

In [ ]:
# =============================================================================
# # BÖLÜM 5: TensorFlow Veri Setlerinin Oluşturulması
# =============================================================================
# Önceki hücrede tanımlanan fonksiyonları kullanarak eğitim ve doğrulama için
# tf.data.Dataset nesnelerini oluşturuyoruz. Bu nesneler, veriyi verimli bir
# şekilde modelin GPU/CPU'suna besleyecektir.

train_dataset = create_dataset(train_df, is_train=True)
val_dataset = create_dataset(val_df, is_train=False)

logging.info(f"Eğitim ve doğrulama için tf.data.Dataset'ler oluşturuldu.")

In [ ]:
# =============================================================================
# # BÖLÜM 6: Özel Değerlendirme Metriği (SNR) ve Callback'i
# =============================================================================
# Yarışmanın değerlendirme metriği olan Sinyal-Gürültü Oranı'nı (SNR)
# hesaplamak için gerekli fonksiyonları ve bu metriği her epoch sonunda
# doğrulama seti üzerinde hesaplayacak olan Keras Callback'ini tanımlıyoruz.

def align_and_get_powers(true_signal, pred_signal, fs):
    """
    Tahmin edilen sinyali, gerçek sinyal ile en iyi şekilde hizalar ve
    sinyal gücü ile hata gücünü hesaplar.
    Hizalama, çapraz korelasyon (cross-correlation) ile yapılır.
    """
    try:
        pred_centered = pred_signal - np.mean(pred_signal)
        true_centered = true_signal - np.mean(true_signal)
        
        max_shift = int(0.2 * fs)
        corr = np.correlate(pred_centered, true_centered, mode='full')
        best_shift = np.argmax(corr) - (len(pred_centered) - 1)
        best_shift = np.clip(best_shift, -max_shift, max_shift)
        
        if best_shift > 0:
            aligned_true = true_centered[:-best_shift]
            aligned_pred = pred_centered[best_shift:]
        else:
            aligned_true = true_centered[-best_shift:]
            aligned_pred = pred_centered[:len(pred_centered) + best_shift]
        
        min_len = min(len(aligned_true), len(aligned_pred))
        if min_len == 0: return 0.0, 1.0
        
        aligned_true, aligned_pred = aligned_true[:min_len], aligned_pred[:min_len]
        
        signal_power = np.sum(aligned_true ** 2)
        error_power = np.sum((aligned_true - aligned_pred) ** 2)
        
        return signal_power, max(error_power, 1e-9)
    except Exception as e:
        logging.error(f"align_and_get_powers içinde hata: {e}")
        return 0.0, 1.0

class SNREvaluationCallback(keras.callbacks.Callback):
    def __init__(self, val_df, full_df):
        super().__init__()
        self.val_df = val_df
        self.full_df = full_df
        self.best_snr = -np.inf

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        cv_scores = []
        
        for rec_id in self.val_df['id'].unique():
            image_path = f"{data_dir}/train/{rec_id}/{rec_id}-0001.png"
            
            # Callback, Python'da (eager mode) çalıştığı için tf.py_function'a gerek yoktur.
            # Python fonksiyonunu doğrudan çağırıyoruz.
            lead_images_tensor, _ = process_path(image_path, signal_path=None)
            
            if tf.shape(lead_images_tensor)[0] != 12:
                logging.warning(f"SNR Hesaplanırken ID {rec_id} için 12 derivasyon bulunamadı, atlanıyor.")
                continue
            
            predicted_sequences = self.model.predict(lead_images_tensor, verbose=0)
            
            try:
                true_ts_df = pd.read_csv(f"{data_dir}/train/{rec_id}/{rec_id}.csv")
                fs = self.full_df[self.full_df['id'] == rec_id].iloc[0]['fs']
            except FileNotFoundError:
                logging.warning(f"SNR Hesaplanırken ID {rec_id} için CSV dosyası bulunamadı, atlanıyor.")
                continue
            
            total_signal_power, total_error_power = 0.0, 0.0
            
            for i, lead_name in enumerate(LEAD_NAMES):
                true_signal = true_ts_df[lead_name].values
                if np.isnan(true_signal).any(): continue
                
                pred_resampled = resample(predicted_sequences[i], len(true_signal))
                signal_power, error_power = align_and_get_powers(true_signal, pred_resampled, fs)
                total_signal_power += signal_power
                total_error_power += error_power
            
            if total_error_power > 0:
                cv_scores.append(10 * np.log10(total_signal_power / total_error_power))
        
        avg_snr = np.mean(cv_scores) if cv_scores else -np.inf
        logs['val_snr'] = avg_snr
        
        if avg_snr > self.best_snr:
            self.best_snr = avg_snr
            print(f" - val_snr: {avg_snr:.4f} (Yeni en iyi skor!)")
        else:
            print(f" - val_snr: {avg_snr:.4f}")

In [ ]:
# =============================================================================
# # BÖLÜM 7: Model Oluşturma Fonksiyonu
# =============================================================================
# Bu hücre, derin öğrenme modelimizi oluşturan fonksiyonu içerir.
# - EfficientNetB0: Önceden ImageNet üzerinde eğitilmiş, güçlü bir evrişimli sinir ağı (CNN) modelidir.
#   Bu modele "temel model" (base model) diyoruz.
# - GlobalAveragePooling2D: Temel modelden gelen özellik haritalarını tek bir vektöre indirger.
# - Dense ve Dropout: Regresyon (zaman serisi tahmini) yapmak için eklediğimiz üst katmanlardır.
#   Dropout, ezberlemeyi önlemeye yardımcı olur.

def build_ecg_model(input_shape, seq_length, dropout_rate):
    """
    EfficientNetB0 tabanlı transfer öğrenme modelini oluşturur.
    """
    # Temel model: EfficientNetB0 (ImageNet ağırlıklarıyla)
    # include_top=False, çünkü kendi sınıflandırma/regresyon katmanlarımızı ekleyeceğiz.
    base_model = EfficientNetB0(
        include_top=False, 
        weights=EFFICIENTNET_WEIGHTS_PATH, 
        input_shape=input_shape
    )
    # Başlangıçta temel modelin ağırlıklarını donduruyoruz.
    base_model.trainable = False
    
    # Modelin Mimarisi
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False) # training=False -> dondurulmuş katmanlar için önemli
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(512, activation='relu')(x)
    # Çıktı katmanı: Tahmin edilecek zaman serisi uzunluğunda, lineer aktivasyonlu.
    outputs = Dense(seq_length, activation='linear')(x)
    
    model = Model(inputs, outputs)
    
    return model, base_model

# Modeli ve temel modeli oluştur
ecg_digitizer_model, base_model = build_ecg_model(
    input_shape=(CONFIG.IMG_HEIGHT, CONFIG.IMG_WIDTH, 3),
    seq_length=CONFIG.SEQ_LENGTH,
    dropout_rate=CONFIG.DROPOUT_RATE
)

# Modeli derle
ecg_digitizer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIG.LEARNING_RATE_HEAD),
    loss='mae' # Ortalama Mutlak Hata (Mean Absolute Error)
)

# Modelin özetini göster
ecg_digitizer_model.summary()

In [ ]:
# =============================================================================
# # BÖLÜM 8: Modelin Eğitilmesi (İki Aşamalı)
# =============================================================================
# Modeli iki aşamada eğiteceğiz:
# 1. Feature Extraction (Özellik Çıkarımı): Sadece modelin tepesine eklediğimiz
#    yeni katmanları (Dense, Dropout) eğitiriz. Bu sırada EfficientNetB0'ın
#    ağırlıkları dondurulmuş kalır. Bu, yeni katmanların temel özellikleri
#    kullanmayı öğrenmesini sağlar.
# 2. Fine-Tuning (İnce Ayar): İlk aşama tamamlandıktan sonra, EfficientNetB0'ın
#    son birkaç katmanını "çözer" ve çok daha düşük bir öğrenme oranı ile
#    tüm modeli eğitmeye devam ederiz. Bu, modelin probleme daha özel
#    özellikler öğrenmesini sağlar.

# --- Gelişmiş Callback'lerin Tanımlanması ---

# 1. EarlyStopping: val_loss 5 epok boyunca iyileşmezse eğitimi durdur.
#    restore_best_weights=True sayesinde en iyi ağırlıklara geri dönülür.
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# 2. ReduceLROnPlateau: val_loss 3 epok boyunca iyileşmezse öğrenme oranını 0.2 ile çarp.
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    mode='min',
    min_lr=1e-7
)

# 3. ModelCheckpoint: En iyi val_snr skorunu elde eden model ağırlıklarını kaydet.
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_snr',
    save_best_only=True,
    mode='max',
    verbose=1
)

# 4. SNR Değerlendirme Callback'i
snr_callback = SNREvaluationCallback(val_df, full_train_df)


# --- AŞAMA 1: Sadece Üst Katmanların Eğitimi ---
logging.info("Aşama 1: Sadece üst katmanların (head) eğitimi başlıyor...")
start_time = time.time()

history_head = ecg_digitizer_model.fit(
    train_dataset,
    epochs=CONFIG.EPOCHS_HEAD,
    validation_data=val_dataset,
    callbacks=[
        snr_callback,
        early_stopping,
        reduce_lr,
        model_checkpoint
    ],
    verbose=1
)

logging.info(f"Aşama 1, {time.time() - start_time:.2f} saniyede tamamlandı.")

# --- AŞAMA 2: İnce Ayar (Fine-Tuning) ---
logging.info("Aşama 2: İnce ayar (fine-tuning) başlıyor...")

# Temel modelin son 20 katmanını çöz (eğitilebilir yap)
base_model.trainable = True
fine_tune_at = -20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Modeli çok daha düşük bir öğrenme oranı ile yeniden derle
ecg_digitizer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIG.LEARNING_RATE_FULL),
    loss='mae'
)

logging.info(f"Temel modelin son {-fine_tune_at} katmanı eğitim için çözüldü.")

start_time_full = time.time()
# Eğitime devam et
history_full = ecg_digitizer_model.fit(
    train_dataset,
    epochs=CONFIG.EPOCHS_HEAD + CONFIG.EPOCHS_FULL, # Toplam epoch sayısını artır
    initial_epoch=history_head.epoch[-1] + 1,      # Kaldığı yerden devam et
    validation_data=val_dataset,
    callbacks=[
        snr_callback,
        early_stopping,
        reduce_lr,
        model_checkpoint
    ],
    verbose=1
)

logging.info(f"Aşama 2, {time.time() - start_time_full:.2f} saniyede tamamlandı.")

# En iyi modeli geri yükle
logging.info("Eğitim boyunca elde edilen en iyi model ağırlıkları yükleniyor...")
ecg_digitizer_model.load_weights('best_model.keras')

In [ ]:
# =============================================================================
# # BÖLÜM 9: Sonuçların Değerlendirilmesi ve Görselleştirilmesi
# =============================================================================
# Bu hücrede, modelin eğitim sürecindeki performansını değerlendiriyoruz.
# Eğitim ve doğrulama kayıp (loss) değerlerini bir grafikte göstererek
# modelin öğrenme sürecini ve olası ezberleme (overfitting) durumunu inceliyoruz.

logging.info("Eğitim ve doğrulama sonuçları özetleniyor...")

# İki eğitim aşamasının geçmişini birleştir
history = {}
for key in history_head.history.keys():
    history[key] = history_head.history[key] + history_full.history.get(key, [])

# Son epoch'taki değerleri al
final_train_loss = history['loss'][-1]
final_val_loss = history['val_loss'][-1]
final_val_snr = history.get('val_snr', [None])[-1]

# Eğitim ve Doğrulama Kayıp Grafiği
plt.figure(figsize=(16, 7))
plt.plot(history['loss'], 'o-', label=f"Eğitim Kaybı (Son: {final_train_loss:.4f})")
plt.plot(history['val_loss'], 'o-', label=f"Doğrulama Kaybı (Son: {final_val_loss:.4f})")
plt.title("Epoch'lara Göre Model Kaybı (MAE)", fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Ortalama Mutlak Hata (Loss)', fontsize=12)
plt.xticks(np.arange(len(history['loss'])))
plt.legend(fontsize=12)
plt.grid(True, linestyle='--')
plt.show()

# Performans değerlendirmesini detaylı bir şekilde yazdır
print("\n" + "="*55)
print(" " * 15 + "PERFORMANS DEĞERLENDİRMESİ")
print("="*55)
print("\n--- Modelin Eğitim Sonu Kayıp Değerleri ---")
print(f"Son Eğitim Kaybı (Train Loss):      {final_train_loss:.4f}")
print(f"Son Doğrulama Kaybı (Val Loss):    {final_val_loss:.4f}")
print("\n" + "-"*55)
print("\n--- Yarışma Metriği Skoru ---")

if final_val_snr is not None and not np.isinf(final_val_snr):
    print(f"LOKAL CV SKORU (Son Epoch Ort. SNR):   {final_val_snr:.4f} dB")
    print(f"En İyi Val-SNR (Eğitim Boyunca):      {snr_callback.best_snr:.4f} dB")
    if snr_callback.best_snr > 0.1:
        print("\nSonuç: Başarılı! Lokal skor, hedef olan 0.1'i aştı.")
    else:
        print("\nSonuç: Hedefe ulaşmak için daha fazla eğitim veya ayarlama gerekebilir.")
else:
    print("\nUYARI: Doğrulama seti üzerinde geçerli bir SNR skoru hesaplanamadı.")
print("="*55 + "\n")

In [ ]:
# =============================================================================
# # BÖLÜM 10: Test Verisi Üzerinde Tahmin ve Sunum Dosyası Oluşturma
# =============================================================================
# Bu son bölümde, eğittiğimiz en iyi modeli kullanarak test seti üzerindeki
# EKG görüntüleri için zaman serisi tahminleri yapıyoruz.
# Tahminleri, yarışmanın istediği formatta bir 'submission.csv' dosyasına kaydediyoruz.

logging.info("Test seti üzerinde tahmin süreci başlıyor.")
predictions = []

for base_id, group in tqdm(test_df.groupby('id'), desc="Test Görüntüleri İşleniyor"):
    image_path = f"{data_dir}/test/{base_id}.png"
    
    # Bu döngü de Python'da çalıştığı için tf.py_function'a gerek yoktur.
    # Fonksiyonu doğrudan çağırıyoruz.
    lead_images_tensor, _ = process_path(image_path, signal_path=None)
    
    if tf.shape(lead_images_tensor)[0] != 12:
        for _, row in group.iterrows():
            for i in range(row['number_of_rows']):
                predictions.append({'id': f"{base_id}_{i}_{row['lead']}", 'value': 0.0})
        continue

    predicted_sequences = ecg_digitizer_model.predict(lead_images_tensor, verbose=0)

    for _, row in group.iterrows():
        lead_name, num_rows_expected = row['lead'], row['number_of_rows']
        lead_index = LEAD_NAMES.index(lead_name)
        
        final_signal = resample(predicted_sequences[lead_index], num_rows_expected)
        
        for i, value in enumerate(final_signal):
            predictions.append({'id': f"{base_id}_{i}_{row['lead']}", 'value': float(value)})

logging.info("Tüm test tahminleri tamamlandı.")

submission_df = pd.DataFrame(predictions)
submission_df.to_csv('submission.csv', index=False)

logging.info("submission.csv dosyası başarıyla oluşturuldu!")
print("\nÖrnek Sunum Dosyası:")
display(submission_df.head(15))